L2-regularized logistic regression for binary or multiclass classification; trains a model (on `train.txt`), optimizes L2 regularization strength on `dev.txt`, and evaluates performance on `test.txt`.  Reports test accuracy with 95% confidence intervals and prints out the strongest coefficients for each class.

In [1]:
from scipy import sparse
from sklearn import linear_model
from collections import Counter
import numpy as np
import operator
import nltk
import math
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler

In [2]:
!python3 -m nltk.downloader punkt

<frozen runpy>:128: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/matthewdworkin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
def load_data(filename):
    X = []
    Y = []
    with open(filename, encoding="utf-8") as file:
        for line in file:
            cols = line.split("\t")
            idd = cols[0]
            label = cols[2].lstrip().rstrip()
            text = cols[3]

            X.append(text)
            Y.append(label)

    return X, Y


In [4]:
class Classifier:

    def __init__(self, feature_method, feature_kwargs, trainX, trainY, devX, devY, testX, testY):
        self.feature_vocab = {}
        self.feature_method = feature_method
        self.feature_kwargs = feature_kwargs
        self.min_feature_count=2
        self.log_reg = None

        self.trainY=trainY
        self.devY=devY
        self.testY=testY
        
        self.trainX = self.process(trainX, training=True)
        self.devX = self.process(devX, training=False)
        self.testX = self.process(testX, training=False)

    # Featurize entire dataset
    def featurize(self, data):
        featurized_data = []
        for text in data:
            feats = self.feature_method(text, **self.feature_kwargs)
            featurized_data.append(feats)
        return featurized_data

    # Read dataset and returned featurized representation as sparse matrix + label array
    def process(self, X_data, training = False):
        
        data = self.featurize(X_data)

        if training:
            fid = 0
            feature_doc_count = Counter()
            for feats in data:
                for feat in feats:
                    feature_doc_count[feat]+= 1

            for feat in feature_doc_count:
                if feature_doc_count[feat] >= self.min_feature_count:
                    self.feature_vocab[feat] = fid
                    fid += 1

        F = len(self.feature_vocab)
        D = len(data)
        X = sparse.dok_matrix((D, F))
        for idx, feats in enumerate(data):
            for feat in feats:
                if feat in self.feature_vocab:
                    X[idx, self.feature_vocab[feat]] = feats[feat]

        return X


    # Train model and evaluate on held-out data
    def train(self):
        scaler = StandardScaler(with_mean=False)  # Use with_mean=False for sparse matrix compatibility
        self.trainX = scaler.fit_transform(self.trainX)
        self.devX = scaler.transform(self.devX)
        self.testX = scaler.transform(self.testX)
        (D,F) = self.trainX.shape
        best_dev_accuracy=0
        best_model=None
        for C in [0.1, 1, 10, 100]:
            self.log_reg = linear_model.LogisticRegression(C = C, max_iter=1000)
            self.log_reg.fit(self.trainX, self.trainY)
            training_accuracy = self.log_reg.score(self.trainX, self.trainY)
            development_accuracy = self.log_reg.score(self.devX, self.devY)
            if development_accuracy > best_dev_accuracy:
                best_dev_accuracy=development_accuracy
                best_model=self.log_reg

#             print("C: %s, Train accuracy: %.3f, Dev accuracy: %.3f" % (C, training_accuracy, development_accuracy))

        self.log_reg=best_model
        

    def test(self):
        return self.log_reg.score(self.testX, self.testY)
        

    def printWeights(self, n=10):

        reverse_vocab=[None]*len(self.log_reg.coef_[0])
        for k in self.feature_vocab:
            reverse_vocab[self.feature_vocab[k]]=k

        # binary
        if len(self.log_reg.classes_) == 2:
              weights=self.log_reg.coef_[0]

              cat=self.log_reg.classes_[1]
              for feature, weight in list(reversed(sorted(zip(reverse_vocab, weights), key = operator.itemgetter(1))))[:n]:
                  print("%s\t%.3f\t%s" % (cat, weight, feature))
              print()

              cat=self.log_reg.classes_[0]
              for feature, weight in list(sorted(zip(reverse_vocab, weights), key = operator.itemgetter(1)))[:n]:
                  print("%s\t%.3f\t%s" % (cat, weight, feature))
              print()

        # multiclass
        else:
          for i, cat in enumerate(self.log_reg.classes_):

              weights=self.log_reg.coef_[i]

              for feature, weight in list(reversed(sorted(zip(reverse_vocab, weights), key = operator.itemgetter(1))))[:n]:
                  print("%s\t%.3f\t%s" % (cat, weight, feature))
              print()

            

In [6]:
import numpy as np
from scipy import sparse
from nltk.tokenize import word_tokenize
import spacy
import gensim.downloader as api
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# Load resources
nlp = spacy.load("en_core_web_sm")  # For NER
word_vectors = api.load("word2vec-google-news-300")  # For Word Embeddings
analyzer = SentimentIntensityAnalyzer()  # For Sentiment Analysis


[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [7]:

def featurize(text, **kwargs):
    feats = {}
    words = word_tokenize(text.lower())

#     # Basic BOW features
    if kwargs.get('bag_of_words', True):
        for word in words:
            feats[f"word_{word}"] = 1

    # Sentiment features
    if kwargs.get('sentiment', False):
        sentiment = analyzer.polarity_scores(text)
        feats.update({
            'sentiment_neg': sentiment['neg'],
            'sentiment_neu': sentiment['neu'],
            'sentiment_pos': sentiment['pos'],
            'sentiment_compound': sentiment['compound']
        })

    # Word embeddings (averaged)
    if kwargs.get('word_embed', False):
        embeddings = [word_vectors[word] for word in words if word in word_vectors]
        if embeddings:
            avg_embedding = np.mean(embeddings, axis=0)
            for i, value in enumerate(avg_embedding):
                feats[f"embedding_{i}"] = value

    # NER features
    if kwargs.get('ner', False):
        doc = nlp(text)
        for ent in doc.ents:
            feats[f"ner_{ent.label_}_{ent.text}"] = 1
            
    if kwargs.get('hyperbolae', False):
        hyperbolic_count = count_hyperbolic_terms(text)
        feats['hyperbolic_count'] = hyperbolic_count
        
    if kwargs.get('topics', False):
        topics = get_document_topics(text)
        for topic_id, weight in topics:
            if weight > 0.5:
                feats[f"topic_{topic_id}"] = weight
            
    return feats



In [8]:
def confidence_intervals(accuracy, n, significance_level):
    critical_value=(1-significance_level)/2
    z_alpha=-1*norm.ppf(critical_value)
    se=math.sqrt((accuracy*(1-accuracy))/n)
    return accuracy-(se*z_alpha), accuracy+(se*z_alpha)

In [9]:
def run(trainingFile, devFile, testFile, featurize_kwargs={}):
    trainX, trainY=load_data(trainingFile)
    devX, devY=load_data(devFile)
    testX, testY=load_data(testFile)
    
    simple_classifier = Classifier(featurize, featurize_kwargs, trainX, trainY, devX, devY, testX, testY)
    simple_classifier.train()
    accuracy=simple_classifier.test()
    
    lower, upper=confidence_intervals(accuracy, len(testY), .95)
    print("Test accuracy for best dev model: %.3f, 95%% CIs: [%.3f %.3f]\n" % (accuracy, lower, upper))

    simple_classifier.printWeights()
    return accuracy
    


# Make train test splits

In [10]:
import pandas as pd
adj_data = pd.read_table("./AP2/adjudicated_data.txt")
adj_data

,1,adjudicated,humor,woman who teaches special-needs children killing it at dinner party
0,2,adjudicated,satire,dnc criticized for overly restrictive debate r...
1,3,adjudicated,irony,custodian taken into custody
2,4,adjudicated,none,mike pence takes oath of office as country's n...
3,5,adjudicated,humor,bedtime story from fucking bible again
4,6,adjudicated,satire,literary theorists admit they still have no id...
...,...,...,...,...
494,496,adjudicated,none,longing to finally visit the cuba of my dreams
495,497,adjudicated,none,trump's evangelical advisors urged him to prot...
496,498,adjudicated,none,flaws in how we evaluate leaders (from kahnema...
497,499,adjudicated,irony,intel unveils oversized novelty processor


In [11]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(adj_data, test_size=100)
train, dev = train_test_split(train, test_size=100)
len(train), len(dev), len(test)

(299, 100, 100)

In [12]:
train.to_csv("./splits/train.txt", sep="\t", index=False)
dev.to_csv("./splits/dev.txt", sep="\t", index=False)
test.to_csv("./splits/test.txt", sep="\t", index=False)

In [13]:
trainingFile = "splits/train.txt"
devFile = "splits/dev.txt"
testFile = "splits/test.txt"

featurize_kwargs = {
    'bag_of_words': True
}
    
run(trainingFile, devFile, testFile, featurize_kwargs)


Test accuracy for best dev model: 0.505, 95% CIs: [0.407 0.602]

humor	0.254	word_man
humor	0.250	word_all
humor	0.248	word_toddler
humor	0.209	word_woman
humor	0.203	word_nothing
humor	0.202	word_road
humor	0.193	word_study
humor	0.172	word_area
humor	0.170	word_,
humor	0.167	word_a

irony	0.249	word_introduces
irony	0.245	word_world
irony	0.242	word_box
irony	0.197	word_river
irony	0.196	word_than
irony	0.196	word_wishes
irony	0.189	word_how
irony	0.188	word_into
irony	0.186	word_up
irony	0.177	word_unveils

none	0.186	word_story
none	0.184	word_problem
none	0.181	word_increases
none	0.179	word_ways
none	0.177	word_on
none	0.172	word_.
none	0.171	word_often
none	0.166	word_record
none	0.165	word_national
none	0.159	word_modern

satire	0.224	word_to
satire	0.207	word_at
satire	0.204	word_fuck
satire	0.195	word_after
satire	0.183	word_saliva
satire	0.181	word_enjoying
satire	0.179	word_bridge
satire	0.168	word_media
satire	0.162	word_features
satire	0.160	word_$



0.504950495049505

## Improving on the baseline

## Sentiment

In [14]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

def sentiment_scores(text):
    analyzer = SentimentIntensityAnalyzer()
    scores = analyzer.polarity_scores(text)
    return scores



## Word Embeddings

In [15]:
import gensim.downloader as api
from nltk.tokenize import word_tokenize

# Load pre-trained Word2Vec model.
word_vectors = api.load("word2vec-google-news-300")

def get_word_embedding(text):
    words = word_tokenize(text.lower())
    embeddings = [word_vectors[word] for word in words if word in word_vectors]
    # Average the word vectors of all words in the text
    if embeddings:
        return sum(embeddings) / len(embeddings)
    else:
        return np.zeros(300)  # Return zero vector if no words have embeddings


## NER

In [16]:
import spacy

# Load an English NLP model
nlp = spacy.load("en_core_web_sm")

def named_entities(text):
    doc = nlp(text)
    return [(ent.text, ent.label_) for ent in doc.ents]


## Topic Features

In [17]:
from gensim.corpora.dictionary import Dictionary
from gensim.models.ldamodel import LdaModel
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re
nltk.download('stopwords')
nltk.download('punkt')

trainingFile = "splits/train.txt"
devFile = "splits/dev.txt"

def preprocess_text(text):
    # Lowercase, remove special characters, and tokenize text
    tokens = word_tokenize(re.sub(r'[\W_]+', ' ', text.lower()))
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words and len(token) > 1]
    return tokens

def read_documents(file_path):
    documents = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            cells = line.split("\t")
            documents.append(preprocess_text(cells[-1]))
            
    return documents

# Read and preprocess the training and dev data
train_documents = read_documents(trainingFile)
dev_documents = read_documents(devFile)

# Combine train and dev documents if you want to use both for training
documents = train_documents + dev_documents

# Assuming 'documents' is a list of tokenized documents (list of list of tokens)
dictionary = Dictionary(documents)
corpus = [dictionary.doc2bow(doc) for doc in documents]

# Train LDA model
lda_model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=10, random_state=42)


def get_document_topics(text):
    tokens = preprocess_text(text)
    # Get the bag-of-words representation for the text
    bow = dictionary.doc2bow(tokens)
    # Retrieve the list of topics with their probabilities
    topics = lda_model.get_document_topics(bow)

    return topics


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/matthewdworkin/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/matthewdworkin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Hyperbolic Terms

In [18]:
hyperbolic_terms = ["unbelievable", "amazing", "incredible", "never", "always", "worst", "best"]

def count_hyperbolic_terms(text):
    words = word_tokenize(text.lower())
    count = sum(1 for word in words if word in hyperbolic_terms)
    return count


In [19]:
from itertools import product
from tqdm import tqdm

params = {
    'bag_of_words': [True, False],
    'sentiment': [True, False],
    'word_embed': [True, False],
    'ner': [True, False],
    'hyperbolae': [True, False],
    'topics': [True, False]
}

keys = params.keys()
values = (params[key] for key in keys)
kwargs_combinations = [dict(zip(keys, combination)) 
                for combination in product(*values) 
                if any(combination)]
# featurize_kwargs = {
#     'bag_of_words': False,
#     'sentiment': False,
#     'word_embed': True,
#     'ner': True, 
#     'hyperbolae': False,
#     'topics': True
# }
    
# run(trainingFile, devFile, testFile, featurize_kwargs)

results = []

for kwarg_combination in tqdm(kwargs_combinations):
    # Run the model with the current combination of features
    accuracy = run(trainingFile, devFile, testFile,kwarg_combination)
    results.append((kwarg_combination, accuracy))

# Find the best performing configuration
best_combination = max(results, key=lambda x: x[1])

print("Best configuration:", best_combination[0])
print("Highest accuracy:", best_combination[1])



  2%|▏         | 1/63 [00:02<02:46,  2.68s/it]

Test accuracy for best dev model: 0.465, 95% CIs: [0.368 0.563]

humor	0.401	word_area
humor	0.389	word_democratic
humor	0.383	word_back
humor	0.380	word_road
humor	0.376	word_man
humor	0.374	word_sitting
humor	0.363	word_life
humor	0.357	word_toddler
humor	0.352	embedding_217
humor	0.352	word_argument

irony	0.413	word_wishes
irony	0.323	word_between
irony	0.312	word_how
irony	0.309	word_up
irony	0.304	word_by
irony	0.297	word_world
irony	0.295	word_wants
irony	0.282	word_box
irony	0.279	word_stephen
irony	0.272	embedding_189

none	0.485	embedding_202
none	0.440	word_'s
none	0.428	word_local
none	0.425	word_bear
none	0.416	embedding_109
none	0.400	embedding_234
none	0.399	embedding_13
none	0.388	word_.
none	0.384	word_was
none	0.382	embedding_198

satire	0.415	embedding_72
satire	0.397	word_media
satire	0.394	topic_8
satire	0.385	word_this
satire	0.381	word_enjoying
satire	0.374	word_during
satire	0.367	word_any
satire	0.365	embedding_293
satire	0.364	word_features
satire	0.361	word_s

  3%|▎         | 2/63 [00:05<02:35,  2.55s/it]

Test accuracy for best dev model: 0.535, 95% CIs: [0.437 0.632]

humor	0.203	word_road
humor	0.198	word_democratic
humor	0.189	word_toddler
humor	0.178	word_some
humor	0.173	embedding_217
humor	0.171	word_sitting
humor	0.170	word_gun
humor	0.167	word_woman
humor	0.155	word_nothing
humor	0.152	embedding_181

irony	0.207	word_wishes
irony	0.193	word_world
irony	0.188	word_between
irony	0.184	word_how
irony	0.175	word_wants
irony	0.169	word_up
irony	0.166	embedding_19
irony	0.162	word_by
irony	0.160	word_box
irony	0.155	word_stephen

none	0.232	word_'s
none	0.227	embedding_202
none	0.201	embedding_109
none	0.191	word_problem
none	0.188	word_.
none	0.183	word_record
none	0.181	embedding_245
none	0.180	word_was
none	0.179	word_bear
none	0.172	word_but

satire	0.233	embedding_72
satire	0.221	word_enjoying
satire	0.210	word_this
satire	0.203	word_talking
satire	0.202	word_features
satire	0.198	word_after
satire	0.194	word_fuck
satire	0.193	word_just
satire	0.179	word_media
satire	0.177	embedd

  5%|▍         | 3/63 [00:07<02:34,  2.58s/it]

...

Test accuracy for best dev model: 0.495, 95% CIs: [0.398 0.593]

humor	0.285	ner_DATE_every day
humor	0.280	ner_CARDINAL_4
humor	0.247	ner_PERSON_god
humor	0.239	ner_NORP_americans
humor	-0.002	topic_5
humor	-0.009	topic_6
humor	-0.056	ner_PERSON_bob corker
humor	-0.068	ner_ORG_gop
humor	-0.081	ner_CARDINAL_three
humor	-0.101	ner_NORP_republican

irony	0.266	ner_PERSON_steven spielberg
irony	0.236	ner_PERSON_clinton
irony	-0.012	ner_GPE_china
irony	-0.054	ner_CARDINAL_three
irony	-0.087	ner_PERSON_bob corker
irony	-0.091	ner_ORG_gop
irony	-0.114	ner_CARDINAL_8
irony	-0.118	ner_ORDINAL_first
irony	-0.119	ner_DATE_2018
irony	-0.125	ner_CARDINAL_two

none	0.564	ner_PERSON_bob corker
none	0.546	ner_PERSON_donald trump
none	0.537	ner_CARDINAL_three
none	0.528	ner_GPE_u.s.
none	0.413	ner_DATE_2018
none	0.412	ner_ORDINAL_first
none	0.404	ner_CARDINAL_8
none	0.403	ner_CARDINAL_two
none	0.387	ner_NORP_republican
none	0.346	ner_GPE_iran

satire	1.766	topic_9
satire	1.691	topic_4
satire	1.679	top

 97%|█████████▋| 61/63 [01:23<00:02,  1.08s/it]

Test accuracy for best dev model: 0.475, 95% CIs: [0.378 0.573]

humor	0.128	ner_DATE_every day
humor	0.128	ner_CARDINAL_4
humor	0.125	ner_PERSON_god
humor	0.067	ner_NORP_americans
humor	-0.033	ner_PERSON_bob corker
humor	-0.039	ner_CARDINAL_three
humor	-0.042	ner_ORG_gop
humor	-0.043	ner_DATE_2018
humor	-0.053	ner_ORDINAL_first
humor	-0.053	ner_NORP_republican

irony	0.145	ner_PERSON_steven spielberg
irony	0.093	ner_PERSON_clinton
irony	-0.024	ner_GPE_china
irony	-0.029	ner_PERSON_bob corker
irony	-0.034	ner_CARDINAL_three
irony	-0.036	ner_CARDINAL_8
irony	-0.036	ner_DATE_2018
irony	-0.037	ner_ORG_gop
irony	-0.045	ner_ORDINAL_first
irony	-0.045	ner_NORP_republican

none	0.246	ner_PERSON_donald trump
none	0.202	ner_GPE_u.s.
none	0.201	ner_PERSON_bob corker
none	0.200	ner_CARDINAL_three
none	0.174	ner_ORDINAL_first
none	0.174	ner_NORP_republican
none	0.174	ner_CARDINAL_two
none	0.168	ner_CARDINAL_8
none	0.156	ner_DATE_2018
none	0.117	ner_GPE_iran

satire	0.187	ner_GPE_china
satire	0.140

100%|██████████| 63/63 [01:24<00:00,  1.33s/it]

Test accuracy for best dev model: 0.505, 95% CIs: [0.407 0.602]

humor	0.242	topic_5
humor	0.156	topic_6
humor	0.056	topic_3
humor	0.036	topic_7
humor	0.028	topic_0
humor	0.009	topic_8
humor	-0.079	topic_2
humor	-0.122	topic_4
humor	-0.153	topic_1
humor	-0.195	topic_9

irony	0.114	topic_9
irony	0.103	topic_3
irony	0.091	topic_1
irony	0.005	topic_4
irony	-0.021	topic_8
irony	-0.032	topic_2
irony	-0.046	topic_6
irony	-0.055	topic_0
irony	-0.114	topic_5
irony	-0.179	topic_7

none	0.134	topic_3
none	0.112	topic_2
none	0.059	topic_7
none	0.024	topic_9
none	-0.031	topic_1
none	-0.056	topic_0
none	-0.097	topic_5
none	-0.104	topic_6
none	-0.115	topic_4
none	-0.175	topic_8

satire	0.231	topic_4
satire	0.187	topic_8
satire	0.093	topic_1
satire	0.084	topic_7
satire	0.082	topic_0
satire	0.057	topic_9
satire	-0.001	topic_2
satire	-0.006	topic_6
satire	-0.032	topic_5
satire	-0.293	topic_3

Best configuration: {'bag_of_words': False, 'sentiment': True, 'word_embed': True, 'ner': True, 'hyperbolae': T

In [20]:
results

[({'bag_of_words': True,
   'sentiment': True,
   'word_embed': True,
   'ner': True,
   'hyperbolae': True,
   'topics': True},
  0.46534653465346537),
 ({'bag_of_words': True,
   'sentiment': True,
   'word_embed': True,
   'ner': True,
   'hyperbolae': True,
   'topics': False},
  0.5346534653465347),
 ({'bag_of_words': True,
   'sentiment': True,
   'word_embed': True,
   'ner': True,
   'hyperbolae': False,
   'topics': True},
  0.46534653465346537),
 ({'bag_of_words': True,
   'sentiment': True,
   'word_embed': True,
   'ner': True,
   'hyperbolae': False,
   'topics': False},
  0.5346534653465347),
 ({'bag_of_words': True,
   'sentiment': True,
   'word_embed': True,
   'ner': False,
   'hyperbolae': True,
   'topics': True},
  0.46534653465346537),
 ({'bag_of_words': True,
   'sentiment': True,
   'word_embed': True,
   'ner': False,
   'hyperbolae': True,
   'topics': False},
  0.5445544554455446),
 ({'bag_of_words': True,
   'sentiment': True,
   'word_embed': True,
   'ner'